## Validación con Prestaciones (V2): KPIs 2025 y Cohorte
### Pagado es booleano (TRUE/FALSE), no monto.
### Los montos abonados no se pueden calcular con este CSV; trabajamos con conteos por paciente y cohorte.

# Setup

In [ ]:
# Paths & imports

from pathlib import Path
import pandas as pd
import numpy as np

# Siempre funciona estando dentro o fuera de /notebooks
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PREST_PATH = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"

assert PREST_PATH.exists(), f"No encuentro: {PREST_PATH}"

pd.set_option("display.max_rows", 50)
pd.set_option("display.max_columns", None)


In [ ]:
# Carga y columnas clave

dfp = pd.read_csv(PREST_PATH, low_memory=False)

# Fechas
fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)

# Año/mes para presupuesto (Fecha) y pago (Fec_Estado; si falta, cae a Fecha)
anio_ppto = fecha.dt.year
mes_ppto  = fecha.dt.month
anio_pago = fec_estado.dt.year.fillna(anio_ppto)
mes_pago  = fec_estado.dt.month.fillna(mes_ppto)

# Pagado booleano robusto
pag_str  = dfp.get('Pagado', pd.Series(index=dfp.index, dtype=object)).astype(str).str.strip().str.upper()
pag_bool = pag_str.map({'TRUE': True, 'FALSE': False, '1': True, '0': False, 'SI': True, 'NO': False}).fillna(False)

# RUT
rut_col = 'RUT' if 'RUT' in dfp.columns else ('RutBeneficiario' if 'RutBeneficiario' in dfp.columns else None)
assert rut_col is not None, "No encuentro columna de RUT."
print("Filas Prestaciones:", len(dfp), "| Cols:", len(dfp.columns), "| RUT:", rut_col)


In [ ]:
# QA rápido de Pagado/Estado

def top_vals(s, n=10):
    return (s.astype(str).str.strip().str.upper()
            .replace({'': '(VACÍO)', 'NAN': '(NULO)'})
            .value_counts().head(n))

print("TOP Pagado:\n", top_vals(dfp['Pagado']))
print("\nTOP Estado:\n", top_vals(dfp.get('Estado', pd.Series(dtype=object))))


# KPIs 2025

In [ ]:
# KPIs por paciente y series

# Detectar una columna de monto de presupuesto (para exigir >0). Si no hay, basta con la existencia.
m_ppto = None
for cand in ['Valor_Total','Precio_Total','Valor_Prest','Precio']:
    if cand in dfp.columns:
        m_ppto = cand; break

if m_ppto is not None:
    monto = pd.to_numeric(dfp[m_ppto], errors='coerce')
    mask_ppto_2025 = anio_ppto.eq(2025) & (monto > 0)
else:
    mask_ppto_2025 = anio_ppto.eq(2025)

mask_pago_2025 = anio_pago.eq(2025) & pag_bool

# KPIs por paciente único
pacientes_ppto_2025 = dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).nunique()
pacientes_pago_2025 = dfp.loc[mask_pago_2025, rut_col].dropna().astype(str).nunique()

# Series mensuales (pacientes únicos por mes)
serie_ppto = (dfp.loc[mask_ppto_2025, [rut_col]]
                .assign(_mes=mes_ppto[mask_ppto_2025].astype('Int64'))
                .dropna().drop_duplicates()
                .groupby('_mes')[rut_col].nunique()
                .reindex(range(1,13), fill_value=0))

serie_pago = (dfp.loc[mask_pago_2025, [rut_col]]
                .assign(_mes=mes_pago[mask_pago_2025].astype('Int64'))
                .dropna().drop_duplicates()
                .groupby('_mes')[rut_col].nunique()
                .reindex(range(1,13), fill_value=0))

# Top convenios entre quienes pagaron en 2025
if 'Convenio' in dfp.columns:
    top_conv_pago = (dfp.loc[mask_pago_2025, [rut_col,'Convenio']]
                       .dropna()
                       .drop_duplicates(rut_col)['Convenio']
                       .astype(str).str.upper()
                       .value_counts().head(10))
else:
    top_conv_pago = pd.Series(dtype='int64')

kpis_2025 = {
    'pacientes_ppto_creado_2025': int(pacientes_ppto_2025),
    'pacientes_con_abono_2025'  : int(pacientes_pago_2025),
    'nota' : 'Pagos 2025 incluyen arrastre de presupuestos antiguos (no linkeamos por ID).'
}

print("KPIs 2025 (conteos por paciente):", kpis_2025)
print("\nPacientes con ppto por mes (2025):\n", serie_ppto)
print("\nPacientes con abono por mes (2025):\n", serie_pago)
print("\nTop convenios (pagaron en 2025):\n", top_conv_pago)


In [ ]:
# Cohorte 2025
# Cohorte: pacientes que crearon Ppto en 2025 (mask_ppto_2025)
cohorte_2025      = set(dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).unique())
pagadores_2025    = set(dfp.loc[mask_pago_2025, rut_col].dropna().astype(str).unique())
pag_en_cohorte    = len(cohorte_2025 & pagadores_2025)
n_cohorte         = len(cohorte_2025)
tasa_cohorte_2025 = (pag_en_cohorte / n_cohorte) if n_cohorte else np.nan

print({
    "cohorte_pacientes_ppto_2025": n_cohorte,
    "pagadores_2025_en_cohorte"  : pag_en_cohorte,
    "tasa_pago_en_misma_cohorte" : None if pd.isna(tasa_cohorte_2025) else round(tasa_cohorte_2025, 3)
})


# Export mini-resumen

In [ ]:
OUT_DIR = ROOT / "data" / "interim"
OUT_DIR.mkdir(parents=True, exist_ok=True)

(serie_ppto.rename("pacientes_ppto_mes_2025")
           .to_csv(OUT_DIR / "prest_serie_ppto_2025.csv", index=True))
(serie_pago.rename("pacientes_pago_mes_2025")
           .to_csv(OUT_DIR / "prest_serie_pago_2025.csv", index=True))

pd.Series(kpis_2025).to_csv(OUT_DIR / "prest_kpis_2025_conteos.csv")
print("Exportados a:", OUT_DIR)


In [ ]:
# === Export NB-06: Cohortes y QA de reglas ===

import pandas as pd
from pathlib import Path

# Definir ruta a carpeta reports
reports_path = Path(ROOT, "reports")
reports_path.mkdir(parents=True, exist_ok=True)

# Construir DataFrame con resultados globales de cohortes
df_prest_val = pd.DataFrame({
    "Corte": ["2025"],
    "Pacientes_con_Ppto": [4564],
    "Pacientes_con_Pago": [4862],
    "Pacientes_con_Ambos": [3951],
    "Tasa_Conversion": [3951 / 4564]  # ~0.91
})

# Exportar a CSV
export_path = reports_path / "nb06_prestaciones_validacion.csv"
df_prest_val.to_csv(export_path, index=False)

print("Archivo exportado en:", export_path)
df_prest_val


In [ ]:
# === Comparativa por Cluster (NB-06) — Celda ÚNICA (FIX flags Ppto/Pago) ===
# Requisitos previos:
#  - dfp (prestaciones) ya cargado
#  - rut_col y fecha_col definidos en celdas anteriores (si no, descomenta las 2 líneas marcadas)

import pandas as pd
import numpy as np
from pathlib import Path
import re

# (Descomenta si NO están definidas arriba)
# rut_col = "RutBeneficiario"   # o "RUT"
# fecha_col = "Fec_Estado"      # o "Fecha", etc.

# 0) ROOT y clusters
ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CLUSTERS_PATH = ROOT / "data" / "processed" / "activos_ids_v2_plus_clustered.csv"
assert CLUSTERS_PATH.exists(), f"No encuentro {CLUSTERS_PATH}."
dfc = pd.read_csv(CLUSTERS_PATH)

# Usaremos baseline para ser consistentes con NB-04
assert "cluster_baseline" in dfc.columns, "La tabla clustered no tiene 'cluster_baseline'."
rut_key_clusters = "RutBeneficiario" if "RutBeneficiario" in dfc.columns else ("RUT" if "RUT" in dfc.columns else None)
assert rut_key_clusters is not None, "No encuentro columna de RUT en la tabla clustered."
dfc = dfc.rename(columns={"cluster_baseline": "Cluster"})

# 1) Preparar año en prestaciones + recorte 2025
dfp["anio"] = pd.to_datetime(dfp[fecha_col], errors="coerce").dt.year
print("Años detectados en dfp:", dfp["anio"].dropna().value_counts().sort_index().tail())
dfp_2025 = dfp[dfp["anio"] == 2025].copy()
print("Filas 2025:", len(dfp_2025))

# 2) Detección ROBUSTA de columnas de Presupuesto / Pago
cols_lower = {c: c.lower() for c in dfp_2025.columns}

# Candidatas por nombre para "ppto" (numéricas o flags)
ppto_name_candidates = [
    "tiene_ppto","tieneppto","flag_ppto","presupuesto","es_presupuesto",
    "montopresupuesto","totalpresupuesto","ppto","presu"
]
# Candidatas por nombre para "pago/abono" (numéricas o flags)
pago_name_candidates = [
    "tiene_pago","tienepago","flag_pago","pagado","pago","abono","abonos",
    "montopago","totalpago","totalpagos","montoabono","totalabonos","abonado"
]

def find_cols(candidates):
    found = []
    for c in dfp_2025.columns:
        lc = c.lower()
        if any(x in lc for x in candidates):
            found.append(c)
    return found

ppto_cols = find_cols(ppto_name_candidates)
pago_cols = find_cols(pago_name_candidates)

print("Cols detectadas ppto:", ppto_cols)
print("Cols detectadas pago:", pago_cols)

# 3) Construcción de flags con múltiples estrategias

# 3.1 Flags desde columnas numéricas/booleanas candidatas
def any_positive(df, cols):
    if not cols:
        return pd.Series(False, index=df.index)
    num = df[cols].apply(pd.to_numeric, errors="coerce").fillna(0)
    if num.shape[1] == 1:
        return (num.iloc[:,0] > 0) | (df[cols[0]].astype(str).str.lower().isin(["si","true","sí"]))
    return (num.sum(axis=1) > 0)

dfp_2025["_has_ppto"] = any_positive(dfp_2025, ppto_cols)
dfp_2025["_has_pago"] = any_positive(dfp_2025, pago_cols)

# 3.2 Si siguen todos False, intentamos derivar por texto en columnas tipo "Estado"
if (not dfp_2025["_has_ppto"].any()) or (not dfp_2025["_has_pago"].any()):
    estado_cols = [c for c in dfp_2025.columns if re.search(r"estado|situacion|tipo", c, flags=re.IGNORECASE)]
    if estado_cols:
        estado_blob = dfp_2025[estado_cols].astype(str).apply(lambda s: s.str.cat(sep="|"), axis=1).str.lower()
        # Heurística por texto (ajusta keywords si usas otros)
        dfp_2025["_has_ppto"] = dfp_2025["_has_ppto"] | estado_blob.str.contains(r"presup|ppto|presu")
        dfp_2025["_has_pago"]  = dfp_2025["_has_pago"]  | estado_blob.str.contains(r"pagad|pago|abon")
        print("Heurística por texto aplicada sobre columnas:", estado_cols)

# 3.3 Copiar EXACTAMENTE la lógica del mini-resumen (por si lo anterior no captura tu caso)
#     - Fecha de presupuesto: 'Fecha' -> anio_ppto
#     - Fecha de pago: 'Fec_Estado' (o cae a 'Fecha') -> anio_pago
#     - Pagado: normalización de 'Pagado'
fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)
anio_ppto  = fecha.dt.year
anio_pago  = fec_estado.dt.year.fillna(anio_ppto)

pag_str  = dfp.get('Pagado', pd.Series(index=dfp.index, dtype=object)).astype(str).str.strip().str.upper()
pag_bool = pag_str.map({'TRUE': True, 'FALSE': False, '1': True, '0': False, 'SI': True, 'NO': False}).fillna(False)

m_ppto = None
for cand in ['Valor_Total','Precio_Total','Valor_Prest','Precio']:
    if cand in dfp.columns:
        m_ppto = cand; break

if m_ppto is not None:
    monto = pd.to_numeric(dfp[m_ppto], errors='coerce')
    mask_ppto_2025 = anio_ppto.eq(2025) & (monto > 0)
else:
    mask_ppto_2025 = anio_ppto.eq(2025)

mask_pago_2025 = anio_pago.eq(2025) & pag_bool

# Refuerzo de flags por fila (OR con lo anterior)
dfp_2025["_has_ppto"] = dfp_2025["_has_ppto"] | mask_ppto_2025.loc[dfp_2025.index].fillna(False)
dfp_2025["_has_pago"]  = dfp_2025["_has_pago"]  | mask_pago_2025.loc[dfp_2025.index].fillna(False)

print("Resumen flags → _has_ppto:", int(dfp_2025["_has_ppto"].sum()),
      "| _has_pago:", int(dfp_2025["_has_pago"].sum()))

# 4) Agregación por paciente (2025)
agg_2025 = (dfp_2025
            .groupby(rut_col)
            .agg(has_ppto=("_has_ppto","max"),
                 has_pago=("_has_pago","max"))
            .reset_index())
agg_2025["has_ambos"] = agg_2025["has_ppto"] & agg_2025["has_pago"]

# 5) Join con clusters
aggc = agg_2025.merge(
    dfc[[rut_key_clusters, "Cluster"]].rename(columns={rut_key_clusters: rut_col}),
    on=rut_col, how="left"
)

# 6) Conversión por cluster
conv = aggc.groupby("Cluster").agg(
    n_pacientes=("has_ppto","size"),
    ppto=("has_ppto","sum"),
    pago=("has_pago","sum"),
    ambos=("has_ambos","sum")
)
conv["tasa_conversion"] = (conv["ambos"] / conv["ppto"]).replace([np.inf, -np.inf], np.nan).round(3)

# 7) Export final
reports_dir = ROOT / "reports" / "entregables"
reports_dir.mkdir(parents=True, exist_ok=True)
conv_out = reports_dir / "nb06_cluster_conversion_2025.csv"
conv.to_csv(conv_out)
print("✔ Export:", conv_out.resolve())

# 8) (Opcional) Mix de prestaciones si existe columna conocida de tipo
tipo_col = None
for c in ["TipoPrestacion","Prestacion","Tipo_Prestacion"]:
    if c in dfp_2025.columns:
        tipo_col = c
        break

if tipo_col:
    mix = (dfp_2025.merge(dfc[[rut_key_clusters, "Cluster"]].rename(columns={rut_key_clusters: rut_col}),
                          on=rut_col, how="left")
           .pivot_table(index="Cluster", columns=tipo_col, values=rut_col, aggfunc="nunique")
           .fillna(0))
    mix_pct = (mix.div(mix.sum(axis=1), axis=0).round(3))
    mix_out = reports_dir / "nb06_cluster_prestaciones_2025.csv"
    mix_pct.to_csv(mix_out)
    print("✔ Export:", mix_out.resolve())
else:
    print("Aviso: no hay columna de tipo de prestación (TipoPrestacion/Prestacion) → se omite mix.")

print("\n[OK] Comparativa por cluster terminada.")



### 🗃️ Histórico V1 (no ejecutar)

In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    fp = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"

    # 1) Cargar e inspeccionar
    dfp = pd.read_csv(fp, low_memory=False)
    print("Cols prestaciones:", list(dfp.columns)[:30])

    # 2) Mapea columnas (ajusta si difiere)
    MAP = {
        "rut":      ["RutBeneficiario","Rut","RUT","IdPaciente","PacienteID"],
        "fecha":    ["Fecha","FechaAtencion","FechaPpto","FechaPresupuesto","FechaRegistro","Fec_Atencion","Fec_Ppto"],
        "tipo":     ["Tipo","TipoPrestacion","Movimiento","Estado","Evento"],
        # Monto total del Ppto y Monto abonado (ajusta si existen)
        "monto_ppto":   ["MontoPresupuestado","TotalPresupuesto","MontoPpto","Total","Valor"],
        "monto_abono":  ["MontoAbonado","Abonado","Pagado","MontoPago","MontoAvance"],
    }

    def pick(d, candidates):
        for c in candidates:
            if c in d.columns: return c
        return None

    col_rut   = pick(dfp, MAP["rut"])
    col_fecha = pick(dfp, MAP["fecha"])
    col_tipo  = pick(dfp, MAP["tipo"])
    col_mp    = pick(dfp, MAP["monto_ppto"])
    col_ma    = pick(dfp, MAP["monto_abono"])

    print("Usando columnas ->", dict(rut=col_rut, fecha=col_fecha, tipo=col_tipo, monto_ppto=col_mp, monto_abono=col_ma))

    # 3) Parse fecha y cortar 2025
    f = pd.to_datetime(dfp[col_fecha], errors="coerce", dayfirst=True)
    dfp["_anio"] = f.dt.year
    dfp["_mes"]  = f.dt.month
    dfp_2025 = dfp[dfp["_anio"].eq(2025)].copy()

    # 4) Derivar KPIs aproximando las tarjetas de PBI
    #   - Define reglas para "ppto creado", "abono" y "avance"
    def flag_contains(s, patterns):
        s = s.astype(str).str.upper()
        pat = "|".join([p.upper() for p in patterns])
        return s.str.contains(pat, regex=True)

    if col_tipo is not None:
        is_ppto  = flag_contains(dfp_2025[col_tipo], ["PRESUPUESTO","PPTO","CREADO"])
        is_abono = flag_contains(dfp_2025[col_tipo], ["ABONO","PAGO","COBRO"])
        is_avance= flag_contains(dfp_2025[col_tipo], ["AVANCE"])
    else:
        # Si no existe 'tipo', nos vamos por montos disponibles
        is_ppto  = dfp_2025[col_mp].notna() if col_mp else pd.Series(False, index=dfp_2025.index)
        is_abono = dfp_2025[col_ma].notna() if col_ma else pd.Series(False, index=dfp_2025.index)
        is_avance= pd.Series(False, index=dfp_2025.index)

    # 5) KPIs (2025)
    kpis = {}

    # Pacientes con ppto creado en 2025
    if col_rut:
        kpis["pacientes_ppto_creado"] = int(dfp_2025.loc[is_ppto, col_rut].nunique())
        kpis["pacientes_con_abono"]   = int(dfp_2025.loc[is_abono, col_rut].nunique())
        kpis["pacientes_con_avance"]  = int(dfp_2025.loc[is_avance, col_rut].nunique())

    # Montos
    if col_mp:
        kpis["monto_total_ppto_2025"] = float(dfp_2025.loc[is_ppto, col_mp].sum())
    if col_ma:
        kpis["monto_total_abonos_2025"] = float(dfp_2025.loc[is_abono, col_ma].sum())

    # % Avance abonado (métricas de PBI suelen ser sum(abonos)/sum(pptos))
    if col_mp and col_ma and kpis.get("monto_total_ppto_2025", 0) > 0:
        kpis["pct_avance_abonado"] = kpis["monto_total_abonos_2025"] / kpis["monto_total_ppto_2025"]

    print("KPIs 2025 estimados:", kpis)

    # 6) Serie por mes (pacientes con ppto por mes)
    if col_rut:
        serie_mes = (dfp_2025.loc[is_ppto, [col_rut, "_mes"]]
                    .dropna()
                    .drop_duplicates()
                    .groupby("_mes")[col_rut].nunique()
                    .reindex(range(1,13), fill_value=0))
        print("Pacientes con ppto creado por mes (2025):")
        print(serie_mes)

    # 7) Top convenios por paciente en 2025
    #    Join con clientes para tomar Empresa/Convenio limpio
    dfc = pd.read_csv(ROOT / "data" / "raw" / "Tab_Clientes(2).csv", low_memory=False)
    # Tomar solo columnas necesarias
    join_cols = ["RutBeneficiario","Empresa"]
    join_cols = [c for c in join_cols if c in dfc.columns]
    dfc_j = dfc[join_cols].copy()

    if col_rut and join_cols:
        dfp_2025_u = dfp_2025.loc[is_ppto, [col_rut]].dropna().drop_duplicates()
        top_conv = (dfp_2025_u.merge(dfc_j, left_on=col_rut, right_on="RutBeneficiario", how="left")
                    .Empresa.astype(str).str.upper().value_counts().head(10))
        print("Top convenios (pacientes con ppto 2025):")
        print(top_conv)


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv", low_memory=False)

    # Parseo mínimo
    dfp['Fecha'] = pd.to_datetime(dfp['Fecha'], errors='coerce', dayfirst=True)
    dfp['_anio'] = dfp['Fecha'].dt.year
    dfp['_mes']  = dfp['Fecha'].dt.month

    # Montos: ppto vs abonos
    m_ppto = 'Valor_Total' if 'Valor_Total' in dfp.columns else ('Precio_Total' if 'Precio_Total' in dfp.columns else None)
    m_abon = 'Pagado' if 'Pagado' in dfp.columns else None
    assert m_abon is not None, "No encuentro columna de abonos (Pagado)."

    # 2025
    d25 = dfp[dfp['_anio'].eq(2025)].copy()

    # Limpieza básica de montos
    for c in [m_ppto, m_abon]:
        if c: d25[c] = pd.to_numeric(d25[c], errors='coerce').fillna(0)

    # KPI por PACIENTE en 2025 (aprox tarjetas PBI)
    agg_pac = (d25
        .groupby('RUT')[[m_ppto, m_abon]]
        .sum(min_count=1)
        .fillna(0)
        .rename(columns={m_ppto:'ppto_2025', m_abon:'abon_2025'}))

    kpis = {
        'pacientes_ppto_creado'   : int((agg_pac['ppto_2025'] > 0).sum()),
        'pacientes_con_abono'     : int((agg_pac['abon_2025'] > 0).sum()),
        'pacientes_ppto_en_avance': int(((agg_pac['abon_2025'] > 0) & (agg_pac['abon_2025'] < agg_pac['ppto_2025'])).sum()),
        'monto_total_ppto_2025'   : float(agg_pac['ppto_2025'].sum()),
        'monto_total_abonos_2025' : float(agg_pac['abon_2025'].sum()),
    }
    kpis['pct_avance_abonado'] = (kpis['monto_total_abonos_2025'] / kpis['monto_total_ppto_2025']) if kpis['monto_total_ppto_2025']>0 else np.nan
    print("KPIs 2025 (aprox paciente):", kpis)

    # Serie mensual (pacientes con presupuesto por mes)
    serie_mes = (d25.groupby(['_mes','RUT'])[m_ppto].sum().reset_index()
                .query(f"{m_ppto} > 0")
                .groupby('_mes')['RUT'].nunique()
                .reindex(range(1,13), fill_value=0))
    print("\nPacientes con ppto por mes (2025):")
    print(serie_mes)

    # Top convenios por paciente (usando Prestaciones directamente)
    top_conv = (d25.groupby('RUT')['Convenio'].agg(lambda s: str(s.dropna().iloc[0]) if len(s.dropna()) else '(EN BLANCO)')
                .value_counts().head(10))
    print("\nTop convenios por paciente (2025, Prestaciones):")
    print(top_conv)

    # (Opcional) Comparar con 'Empresa' de Clientes:
    dfc = pd.read_csv(ROOT / "data" / "raw" / "Tab_Clientes(2).csv", low_memory=False)
    top_conv_emp = (d25[['RUT']].drop_duplicates()
                    .merge(dfc[['RutBeneficiario','Empresa']], left_on='RUT', right_on='RutBeneficiario', how='left')
                    .Empresa.astype(str).str.upper().value_counts().head(10))
    print("\nTop convenios por paciente (2025, via Empresa de Clientes):")
    print(top_conv_emp)


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # Normalización de montos CLP con miles '.' y decimales ','
    import pandas as pd
    def parse_clp(s):
        if s is None: 
            return pd.Series(dtype='float64')
        return pd.to_numeric(
            s.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )

    d25_norm = d25.copy()
    if m_abon:  d25_norm[m_abon] = parse_clp(d25_norm[m_abon])
    if m_ppto:  d25_norm[m_ppto] = parse_clp(d25_norm[m_ppto])

    agg_pac_n = (d25_norm
                .groupby('RUT')[[m_ppto, m_abon]]
                .sum(min_count=1).fillna(0)
                .rename(columns={m_ppto:'ppto_2025', m_abon:'abon_2025'}))

    kpis_n = {
        'pacientes_ppto_creado'   : int((agg_pac_n['ppto_2025'] > 0).sum()),
        'pacientes_con_abono'     : int((agg_pac_n['abon_2025'] > 0).sum()),
        'pacientes_ppto_en_avance': int(((agg_pac_n['abon_2025'] > 0) & (agg_pac_n['abon_2025'] < agg_pac_n['ppto_2025'])).sum()),
        'monto_total_ppto_2025'   : float(agg_pac_n['ppto_2025'].sum()),
        'monto_total_abonos_2025' : float(agg_pac_n['abon_2025'].sum()),
    }
    kpis_n['pct_avance_abonado'] = (kpis_n['monto_total_abonos_2025']/kpis_n['monto_total_ppto_2025']
                                    if kpis_n['monto_total_ppto_2025']>0 else None)
kpis_n


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # DIAGNÓSTICO: ¿Los abonos 2025 están fechados en Fec_Estado?
    import pandas as pd
    from pathlib import Path

    ROOT = Path.cwd().parent if Path.cwd().name=="notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT/"data"/"raw"/"ETL_vPrestaciones (2).csv", low_memory=False)

    def parse_clp_series(s):
        return pd.to_numeric(
            s.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False),
            errors='coerce'
        )

    fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
    fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)
    pag        = parse_clp_series(dfp['Pagado']) if 'Pagado' in dfp.columns else pd.Series(dtype='float64')
    estado     = dfp.get('Estado', pd.Series(index=dfp.index, dtype=str)).astype(str).str.upper()

    print("Non-null Pagado:", int(pag.notna().sum()), " | Suma total (todas fechas):", float(pag.fillna(0).sum()))

    by_year_fecha = pag.groupby(fecha.dt.year).sum(min_count=1).dropna().sort_index()
    by_year_fest  = pag.groupby(fec_estado.dt.year).sum(min_count=1).dropna().sort_index()
    print("\nSuma Pagado por año (usando Fecha):\n", by_year_fecha.tail(6))
    print("\nSuma Pagado por año (usando Fec_Estado):\n", by_year_fest.tail(6))

    print("\nTop Estado con pago>0:\n", estado[pag>0].value_counts().head(10))

    mask_2025_fest = fec_estado.dt.year.eq(2025)
    print("\n2025 por Fec_Estado → filas con pago>0:", int((pag[mask_2025_fest]>0).sum()),
        " | suma:", float(pag[mask_2025_fest].sum()))


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    from pathlib import Path
    import pandas as pd

    ROOT = Path.cwd().parent if Path.cwd().name=="notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT/"data"/"raw"/"ETL_vPrestaciones (2).csv", low_memory=False)

    def top_vals(s, n=20):
        return (s.astype(str).str.strip().str.upper()
                .replace({'': '(VACÍO)', 'NAN': '(NULO)'})
                .value_counts().head(n))

    print(">>> TOP valores 'Pagado':")
    print(top_vals(dfp['Pagado']))

    print("\n>>> TOP valores 'Estado':")
    print(top_vals(dfp['Estado']))

    print("\nMuestra cruda Pagado (primeras 10 celdas):")
    print(dfp['Pagado'].head(10).tolist())


In [ ]:
# --- OBSOLETO: bloque V1, no ejecutar ---
if False:
    pass

    # KPIs 2025 basados en Pagado (booleano) — conteos por paciente
    from pathlib import Path
    import pandas as pd
    import numpy as np

    ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
    dfp = pd.read_csv(ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv", low_memory=False)

    # --- Parse de fechas
    fecha      = pd.to_datetime(dfp.get('Fecha'), errors='coerce', dayfirst=True)
    fec_estado = pd.to_datetime(dfp.get('Fec_Estado'), errors='coerce', dayfirst=True)

    # Año/mes de "presupuesto/registro" y de "pago"
    anio_ppto = fecha.dt.year
    mes_ppto  = fecha.dt.month

    anio_pago = fec_estado.dt.year.fillna(anio_ppto)  # si no hay Fec_Estado, caer a Fecha
    mes_pago  = fec_estado.dt.month.fillna(mes_ppto)

    # --- Pagado como booleano robusto
    pag_bool = (dfp['Pagado']
                .astype(str).str.strip().str.upper()
                .map({'TRUE': True, 'FALSE': False})
                .fillna(False))

    # --- Montos de referencia del "presupuesto" (solo para identificar que hay registro)
    m_ppto = None
    for cand in ['Valor_Total', 'Precio_Total', 'Valor_Prest', 'Precio']:
        if cand in dfp.columns:
            m_ppto = cand
            break

    if m_ppto is None:
        # Si no hay montos, usamos la mera existencia de filas como "registro de ppto"
        tiene_ppto_2025 = anio_ppto.eq(2025)
    else:
        # Consideramos "ppto" si el monto es > 0
        monto = pd.to_numeric(dfp[m_ppto], errors='coerce')
        tiene_ppto_2025 = anio_ppto.eq(2025) & (monto > 0)

    # --- Flags 2025
    tiene_pago_2025 = anio_pago.eq(2025) & pag_bool

    # --- KPIs por PACIENTE
    rut_col = 'RUT' if 'RUT' in dfp.columns else 'RutBeneficiario'
    assert rut_col in dfp.columns, "No encuentro columna de RUT para identificar pacientes."

    pacientes_ppto_2025 = dfp.loc[tiene_ppto_2025, rut_col].dropna().astype(str).nunique()
    pacientes_pago_2025 = dfp.loc[tiene_pago_2025, rut_col].dropna().astype(str).nunique()

    # Serie mensual (n° pacientes únicos por mes)
    serie_ppto = (dfp.loc[tiene_ppto_2025, [rut_col]]
                    .assign(_mes=mes_ppto[tiene_ppto_2025].astype('Int64'))
                    .dropna()
                    .drop_duplicates()
                    .groupby('_mes')[rut_col].nunique()
                    .reindex(range(1,13), fill_value=0))

    serie_pago = (dfp.loc[tiene_pago_2025, [rut_col]]
                    .assign(_mes=mes_pago[tiene_pago_2025].astype('Int64'))
                    .dropna()
                    .drop_duplicates()
                    .groupby('_mes')[rut_col].nunique()
                    .reindex(range(1,13), fill_value=0))

    # Top convenios entre quienes pagaron en 2025
    if 'Convenio' in dfp.columns:
        top_conv_pago = (dfp.loc[tiene_pago_2025, ['RUT','Convenio']]
                        .dropna()
                        .drop_duplicates('RUT')['Convenio']
                        .astype(str).str.upper()
                        .value_counts().head(10))
    else:
        top_conv_pago = pd.Series(dtype='int64')

    kpis_bool = {
        'pacientes_ppto_creado_2025': int(pacientes_ppto_2025),
        'pacientes_con_abono_2025'  : int(pacientes_pago_2025),
        'tasa_pacientes_con_abono'  : (int(pacientes_pago_2025) / int(pacientes_ppto_2025)) if pacientes_ppto_2025 else np.nan,
    }

    print("KPIs 2025 (conteos por paciente, Pagado booleano):", kpis_bool)
    print("\nPacientes con ppto por mes (2025):\n", serie_ppto)
    print("\nPacientes con abono por mes (2025):\n", serie_pago)
    print("\nTop convenios (pacientes que pagaron en 2025):\n", top_conv_pago)


In [ ]:
# --- Chequeo Titular vs Beneficiario (opcional) ---
from pathlib import Path
import pandas as pd

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
CLIENTES_PATH = ROOT / "data" / "raw" / "Tab_Clientes(2).csv"
if CLIENTES_PATH.exists():
    dfc = pd.read_csv(CLIENTES_PATH, low_memory=False)
    if {'RutTitular','RutBeneficiario'}.issubset(dfc.columns):
        a = dfc['RutTitular'].astype(str)
        b = dfc['RutBeneficiario'].astype(str)
        prop_iguales = (a == b).mean()
        print("Proporción titular == beneficiario:", round(prop_iguales, 3))

        dist = (dfc.groupby('RutTitular')['RutBeneficiario']
                  .nunique().describe(percentiles=[.5,.9,.95]))
        print("Beneficiarios distintos por titular (describe):\n", dist)
    else:
        print("No están ambas columnas en Clientes.")
else:
    print("No encuentro", CLIENTES_PATH)


In [ ]:
# === QA AUTOCONTENIDO: carga, parsing y pruebas de alineación ===
from pathlib import Path
import pandas as pd
import numpy as np

ROOT = Path.cwd().parent if Path.cwd().name == "notebooks" else Path.cwd()
PREST_PATH = ROOT / "data" / "raw" / "ETL_vPrestaciones (2).csv"
assert PREST_PATH.exists(), f"No encuentro: {PREST_PATH}"

# --- Carga base de Prestaciones
dfp = pd.read_csv(PREST_PATH, low_memory=False)

# Columnas clave (robustas a mayúsculas/minúsculas)
cols = {c.lower(): c for c in dfp.columns}
rut_col     = cols.get('rut', 'RUT' if 'RUT' in dfp.columns else None)
fecha_col   = cols.get('fecha', 'Fecha' if 'Fecha' in dfp.columns else None)
fecest_col  = cols.get('fec_estado', 'Fec_Estado' if 'Fec_Estado' in dfp.columns else None)
estado_col  = cols.get('estado', 'Estado' if 'Estado' in dfp.columns else None)
pagado_col  = cols.get('pagado', 'Pagado' if 'Pagado' in dfp.columns else None)
ot_col      = cols.get('ot', 'OT' if 'OT' in dfp.columns else None)

needed = [rut_col, fecha_col, estado_col, pagado_col]
assert all(x in dfp.columns for x in needed), f"Faltan columnas mínimas: {needed}"

# --- Parseo de fechas
dfp['_fecha']     = pd.to_datetime(dfp[fecha_col], errors='coerce', dayfirst=True)
dfp['_fecestado'] = pd.to_datetime(dfp.get(fecest_col), errors='coerce', dayfirst=True) if fecest_col in dfp.columns else pd.NaT

# --- Años derivados
dfp['_anio_fecha']     = dfp['_fecha'].dt.year
dfp['_anio_fecestado'] = dfp['_fecestado'].dt.year

# --- Booleano de pago desde 'Pagado' (TRUE/FALSE u otras variantes)
def to_bool(x):
    if pd.isna(x): return False
    s = str(x).strip().upper()
    if s in {'TRUE','1','SI','SÍ','YES'}: return True
    if s in {'FALSE','0','NO'}: return False
    return False

dfp['_pag_bool'] = dfp[pagado_col].map(to_bool)

# --- QA-1: “ppto 2025” solo en ciertos ESTADOS (DIAGNOSTICADA/INICIADA)
estados_ppto = {"DIAGNOSTICADA", "INICIADA"}
mask_ppto_2025 = (dfp['_anio_fecha'] == 2025) & (dfp[estado_col].astype(str).str.upper().isin(estados_ppto))
p_pp_to = dfp.loc[mask_ppto_2025, rut_col].dropna().astype(str).nunique()
print("Pacientes ppto 2025 (solo estados DIAGNOSTICADA/INICIADA):", p_pp_to)

# --- QA-2: ¿y si cuentan OT (órdenes) en vez de paciente?
if ot_col in dfp.columns:
    ot_ppto_2025 = dfp.loc[dfp['_anio_fecha'] == 2025, ot_col].dropna().astype(str).nunique()
    # pagos en 2025 por Fec_Estado + Pagado True
    mask_pago_2025 = (dfp['_anio_fecestado'] == 2025) & (dfp['_pag_bool'])
    ot_pago_2025 = dfp.loc[mask_pago_2025, ot_col].dropna().astype(str).nunique()
    print("OT con ppto 2025:", ot_ppto_2025, " | OT con pago 2025:", ot_pago_2025)
else:
    print("No hay columna OT en Prestaciones; QA-2 no aplica.")

# --- (Extra) Tus métricas “por paciente” para comparar rápidamente:
pac_ppto_2025 = dfp.loc[dfp['_anio_fecha'] == 2025, rut_col].dropna().astype(str).nunique()
pac_pago_2025 = dfp.loc[(dfp['_anio_fecestado'] == 2025) & dfp['_pag_bool'], rut_col].dropna().astype(str).nunique()
print("Paciente único: ppto_2025 =", pac_ppto_2025, " | pago_2025 =", pac_pago_2025)



In [ ]:
# === Generar Guía de Mejora de Datos: .md -> .html -> (opcional) .pdf ===

from pathlib import Path               # Manejo seguro de rutas multiplataforma
import textwrap                        # Para quitar indentaciones indeseadas
import datetime as dt                  # Para sellar fecha si quieres
import sys                             # Para detectar si hay módulos opcionales

# 1) Definimos ROOT (igual que en tus notebooks: sube un nivel desde /notebooks)
ROOT = Path.cwd().parent               # Carpeta raíz del proyecto
reports_dir = ROOT / "reports"         # Carpeta pública de reportes
reports_dir.mkdir(parents=True, exist_ok=True)  # Crea /reports si no existe

# 2) Definimos rutas de salida para los archivos
md_path   = reports_dir / "guia_mejora_datos.md"   # Archivo markdown final
html_path = reports_dir / "guia_mejora_datos.html" # Export en HTML
pdf_path  = reports_dir / "guia_mejora_datos.pdf"  # Export en PDF (opcional)

# 3) Contenido Markdown (pegamos el bloque que te pasé arriba)
md_content = """
# Guía de Mejora de Datos — Portal Ortodoncia
**Fecha:** 06-sep-2025  
**Autor:** Santiago Tupper  
**Contexto:** Proyecto de segmentación de pacientes con clustering (K=3) — foco en reproducibilidad, interpretabilidad y privacidad.

---

## 1) Objetivo
Alinear reglas y prioridades para **mejorar la calidad de datos** que alimenta el análisis (Tab_Clientes y Prestaciones), con el fin de:
- aumentar la confiabilidad de métricas (p. ej., % cumplimiento, recencia),
- facilitar segmentaciones accionables (retención, reactivación, depuración),
- y reducir retrabajo entre 'dashboard.pbix' y Analytics.

---

## 2) Alcance
- **Incluye:** Fechas (histórico vs planificación), `DiasDesdeUltimaVisita`, KPIs de presupuesto, presencias 15d–6m, geografía (Comuna/Región), Empresa/Convenio, vínculo ppto↔pago (Prestaciones).
- **No incluye (por ahora):** detalle clínico por prestación, rendimiento por profesional, costos.

---

## 3) Hallazgos de calidad (resumen)
**Fechas**
- Histórico en ISO; planificación en **DD/MM/AAAA** con **fechas futuras** (baja incidencia).  
- Se crearon flags de planificación y `dias_hasta` (mediana ≈ 65 días; máx ≈ 114).  
- Acción: mantener doble parsing y flags; consolidar diccionario de campos fecha.

**`DiasDesdeUltimaVisita`**
- Validado vs recálculo; **offset sistemático +11 días** (100% positivo en distribución).  
- Decisión: usar el **valor del sistema** y documentar offset; revisar origen junto a TI.

**KPIs de presupuestos**
- Colas largas (outliers) en montos → se usa **`log1p`**.  
- `TicketPromPpto` incluido; **NaN→0 solo para el modelo** (interpretación “sin actividad”).  
- Acción: estandarizar reglas de KPIs y denominadores protegidos.

**Presencias de atención (uso real)**
- Ventanas con mejor señal: **15d, 1m, 3m, 6m**.  
- Acción: mantener solo estas cuatro; descartar 1a/2a en el set de modelado.

**Geografía**
- `Comuna_grp` (Top-N + Otras/Infreq + Sin Comuna) y `Region` con dummies.  
- >95% de pacientes en **Región Metropolitana**; C2 tiene **registros incompletos** (Sin Región 30%).  
- Acción: reforzar captura de Región/Comuna y mapeo maestro.

**Empresa/Convenio**
- Cobertura desigual; **no aportó separación adicional** en clustering (A/B).  
- Acción: mantener como metadato; no forzar su uso en el modelo si no mejora calidad.

**Prestaciones (cohortes 2025)**
- ppto=4.564, pago=4.862, ambos=3.951 → **tasa 91%**.  
- Acción: fijar reglas de estados válidos (DIAGNOSTICADA/INICIADA) y consistencia ppto↔OT.

---

## 4) Reglas acordadas (versión aplicable)
- **Fechas:** parseo dual (ISO vs DD/MM/AAAA) + flags de planificación; conservar `dias_hasta`.  
- **DiasDesdeUltimaVisita:** usar valor del sistema; registrar offset +11 en documentación técnica.  
- **KPIs:** `log1p` en montos; salvaguardas de denominador; `NaN→0` solo para features de modelado.  
- **Ventanas:** solo presencias **15d, 1m, 3m, 6m**.  
- **Geografía:** mantener `Comuna_grp` y `Region` (one-hot); controlar “Sin Comuna/Región”.  
- **Empresa/Convenio:** no se usa para separar clusters (respaldo A/B); se conserva como contexto.  
- **Prestaciones:** cohortes por año; estados válidos DIAGNOSTICADA/INICIADA; controles ppto↔OT.

---

## 5) Recomendaciones priorizadas
| Prioridad | Recomendación | Detalle | Éxito (cómo se mide) |
|---|---|---|---|
| **MUST** | Normalizar fechas | Diccionario de campos + validación automática de formato | 0% errores de parseo; flags coherentes |
| **MUST** | Revisar origen de offset +11 días | Revisión con TI del cálculo de `DiasDesdeUltimaVisita` | Offset documentado/ajustado |
| **MUST** | Estandarizar KPIs | Reglas compartidas con 'dashboard.pbix' (denominadores, `log1p`) | Discrepancias < ±1% |
| **SHOULD** | Completar geografía | Reducción de “Sin Comuna/Región”; maestro de comunas | <2% “Sin Comuna/Región” |
| **SHOULD** | Consolidar vínculo ppto↔pago | QA por OT y por paciente/año | Consistencias > 98% |
| **COULD** | Reevaluar Empresa/Convenio | Mejoras de calidad + nueva A/B | Sólo se usa si mejora métricas |

---

## 6) Roadmap sugerido
1. **Semana 1–2:** Normalización de fechas y documentación del offset de `DiasDesdeUltimaVisita`.  
2. **Semana 3:** Reglas de KPIs consensuadas entre Analytics y 'dashboard.pbix' (one-pager).  
3. **Semana 4–5:** Limpieza geográfica y maestro de comunas/regiones.  
4. **Semana 6:** QA ppto↔pago (Prestaciones) y reporte de consistencia.

---

## 7) Métricas de éxito
- Errores de parseo de fechas = 0%.  
- Discrepancia de KPIs ('dashboard.pbix' vs Analytics) < ±1%.  
- “Sin Región/Comuna” < 2%.  
- Consistencia ppto↔pago > 98%.  
- Trazabilidad reproducible de IDs (RutBeneficiario, RutTitular).

---

## 8) Anexos
- **Metodología:** pipeline mixto de escalado (Standard/Robust/passthrough).  
- **Clustering:** K=3; baseline basta (Empresa no agrega separación).  
- **Archivos relacionados:**  
  - `/reports/nb04_insights_executive.pdf`  
  - `/reports/nb04_dashboard_vs_clustering.csv`  
  - `/reports/nb06_prestaciones_validacion.csv`

---
"""

# 4) Guardamos el .md con el contenido anterior
md_path.write_text(textwrap.dedent(md_content), encoding="utf-8")  # Quita indentación y guarda UTF-8
print(f"[OK] Markdown guardado en: {md_path}")

# 5) Convertimos Markdown -> HTML (usando la librería estándar 'markdown' si está instalada)
try:
    import markdown  # Intentamos usar 'markdown' (pip install markdown)
    html_body = markdown.markdown(md_path.read_text(encoding="utf-8"), extensions=["tables"])  # Soporta tablas
except Exception as e:
    # Si no está instalada, dejamos el contenido envuelto en <pre> para no bloquear el flujo
    print("[WARN] Paquete 'markdown' no disponible. Generaré HTML básico de emergencia.")
    raw = md_path.read_text(encoding="utf-8")
    html_body = f"<pre>{raw}</pre>"

# 6) Envolvemos el HTML en una plantilla sencilla (CSS mínimo legible)
html_template = f"""<!doctype html>
<html lang="es">
<head>
<meta charset="utf-8">
<meta name="viewport" content="width=device-width, initial-scale=1">
<title>Guía de Mejora de Datos — Portal Ortodoncia</title>
<style>
 body {{ font-family: -apple-system, BlinkMacSystemFont, "Segoe UI", Roboto, Arial, "Helvetica Neue", sans-serif; line-height: 1.45; margin: 40px; }}
 h1,h2,h3 {{ margin-top: 1.2em; }}
 code, pre {{ background: #f6f8fa; padding: 2px 4px; border-radius: 4px; }}
 table {{ border-collapse: collapse; width: 100%; margin: 1em 0; }}
 th, td {{ border: 1px solid #ddd; padding: 8px; }}
 th {{ background: #fafafa; }}
 hr {{ border: none; border-top: 1px solid #eee; margin: 24px 0; }}
</style>
</head>
<body>
{html_body}
</body>
</html>"""

# 7) Guardamos el .html final
html_path.write_text(html_template, encoding="utf-8")
print(f"[OK] HTML guardado en: {html_path}")

# 8) (Opcional) HTML -> PDF con pdfkit + wkhtmltopdf si está disponible
try:
    import pdfkit  # Requiere: pip install pdfkit  y tener 'wkhtmltopdf' instalado en el sistema
    pdfkit.from_file(str(html_path), str(pdf_path))
    print(f"[OK] PDF exportado en: {pdf_path}")
except Exception as e:
    # Si no se puede generar el PDF automáticamente, damos una instrucción clara
    print("[INFO] No se pudo generar el PDF automáticamente.")
    print("      - Opción A: instala dependencias y reintenta:")
    print("        pip install markdown pdfkit")
    print("        Instala 'wkhtmltopdf' (macOS: brew install wkhtmltopdf)")
    print("      - Opción B: abre el HTML y usa 'Imprimir' -> 'Guardar como PDF'.")
    print(f"      HTML listo en: {html_path}")
